In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from IPython.display import  clear_output
import time

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

DEFAULT_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

# Configure 4-bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4",  # Normalized float 4-bit (recommended)
    bnb_4bit_compute_dtype=torch.float16,  
    bnb_4bit_use_double_quant=True  # Improves performance by applying second quantization
)

model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    quantization_config=bnb_config,
    # torch_dtype=torch.bfloat16,
    device_map=device,
)

tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = 128001

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def update_chat_history(ai_response, user_query, chat_history):
    chat_history.extend([{"role": "assistant", "content": ai_response},
                         {"role": "user", "content": user_query}])
    return chat_history

# conversation = update_chat_history(ai_response=processed_text,user_query='who is Narendra Modi', chat_history=conversation)

In [27]:

conversation = [
    {   "role": "system", "content": f''' 
        Environment: ipython                                                                               
        Tools: brave_search, wolfram_alpha                                                                                                   
        You are a helpful assistant. '''},
    
    {"role": "user", "content": f'''write a python code to count r in strawberries'''},
]

def llm_generate_response(conversation, model, tokenizer, device):
    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            do_sample=False,
            max_new_tokens=200
        )
    processed_text = tokenizer.decode(output[0][len(inputs.input_ids[0])+4:], skip_special_tokens=True)
    return processed_text

In [11]:
def count_r_in_strawberries(strawberries):
    count = 0
    for char in strawberries:
        if char.lower() == 'r':
            count += 1
    return count

# Test the function
print(count_r_in_strawberries("strawberries")) 

3


In [ ]:
import 

password = getpass.getpass('Enter your password')

print('Your password is: ' + password)

Your password is: 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


In [54]:
import os, arxiv, PyPDF2,getpass
from tavily import TavilyClient

api_key = getpass.getpass('Enter your travily API key')
tavily_client = TavilyClient(api_key)

In [15]:
response = tavily_client.search(
    query="who is mohan dash"
)
print(response)

{'query': 'who is mohan dash', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Mohan Dash - Consultant- Core Business Operation - LinkedIn', 'url': 'https://in.linkedin.com/in/mohan-dash-634885148', 'content': "View Mohan Dash's profile on LinkedIn, a professional community of 1 billion members. AWS cloud | Devops | Python · A Python Django Developer , experience in analysis design, development and implementing e-commerce application, Databases and Client-server Architecture using Python /Django Web Framework · Deloitte · National Institute of", 'score': 0.7434107, 'raw_content': None}, {'title': '\u202aBalyogi Mohan Dash\u202c - \u202aGoogle Scholar\u202c', 'url': 'https://scholar.google.com/citations?user=jzcIElIAAAAJ&hl=en', 'content': 'Balyogi Mohan Dash. PhD in Artificial Intelligence. Verified email at univ-lille.fr - Homepage. Fault Diagnosis and Prognosis Machine Learning Renewable Energy Green Hydrogen. ... BM Dash, BO Bouamama, M Boukerdja, K

In [17]:
response.keys()

dict_keys(['query', 'follow_up_questions', 'answer', 'images', 'results', 'response_time'])

In [20]:
response['results']

[{'title': 'Mohan Dash - Consultant- Core Business Operation - LinkedIn',
  'url': 'https://in.linkedin.com/in/mohan-dash-634885148',
  'content': "View Mohan Dash's profile on LinkedIn, a professional community of 1 billion members. AWS cloud | Devops | Python · A Python Django Developer , experience in analysis design, development and implementing e-commerce application, Databases and Client-server Architecture using Python /Django Web Framework · Deloitte · National Institute of",
  'score': 0.7434107,
  'raw_content': None},
 {'title': '\u202aBalyogi Mohan Dash\u202c - \u202aGoogle Scholar\u202c',
  'url': 'https://scholar.google.com/citations?user=jzcIElIAAAAJ&hl=en',
  'content': 'Balyogi Mohan Dash. PhD in Artificial Intelligence. Verified email at univ-lille.fr - Homepage. Fault Diagnosis and Prognosis Machine Learning Renewable Energy Green Hydrogen. ... BM Dash, BO Bouamama, M Boukerdja, KM Pekpe. 2022 23rd International Middle East Power Systems Conference (MEPCON), 1-7, 202

In [21]:
def query_for_two_papers(paper_1:str , paper_2: str) -> None :
     return [tavily_client.search(f"arxiv id of {paper_1}"), tavily_client.search(f"arxiv id of {paper_2}")]

In [22]:
search_results = query_for_two_papers("llama 3.1", "BERT")

NameError: name 'server_results' is not defined

In [57]:
MAIN_SYSTEM_PROMPT = """
Environment: iPython
Cutting Knowledge Date: December 2023
Today Date: 15 September 2024

# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available

You have access to the following functions:



Use the function 'get_arxiv_ids' to: Given a dict of websearch queries, use a LLM to return JUST the arxiv ID, which is otherwise harder to extract
{
  "name": "get_arxiv_ids",
  "description": "Use the dictionary returned from query_for_two_papers to ask a LLM to extract the arxiv IDs",
  "parameters": {
    "web_results": {
      "param_type": "dictionary",
      "description": "dictionary of search result for a query from the previous function",
      "required": true
    },
  }
}


If a you choose to call a function ONLY reply in the following format:
<{start_tag}={function_name}>{parameters}{end_tag}
where

start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`

Here is an example,
<function=example_function_name>{"example_name": "example_value"}</function>

Reminder:
- When user is asking for a question that requires your reasoning, DO NOT USE OR FORCE a function call
- Even if you remember the arxiv ID of papers from input, do not put that in the query_two_papers function call, pass the internet look up query
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- When returning a function call, don't add anything else to your response

"""

In [56]:
def query_for_two_papers(paper_1:str , paper_2: str) -> None :
     return [tavily_client.search(f"arxiv id of {paper_1}"), tavily_client.search(f"arxiv id of {paper_2}")]
 
search_results = query_for_two_papers("llama 3.1", "BERT")

In [59]:
chat_history = [{"role" : "system","content" : '''Given this input, give me the arXiv ID of the papers. The input has the query and web results. DO NOT WRITE ANYTHING ELSE IN YOUR RESPONSE: ONLY THE ARXIV ID ONCE, the web search will have it repeated multiple times, just return the it once and where its actually the arxiv ID'''},
                {"role" : "user","content" : f'''Here are the search results for the first paper, extract the arxiv ID {search_results[0]}'''}]
llm_output = llm_generate_response(chat_history, model, tokenizer, device)

print(llm_output)

/home/mohan.dash/miniconda3/envs/diffuser_data_generation/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mohan.dash/miniconda3/envs/diffuser_data_generation/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


2407.21783


In [46]:
search_results[0]

{'query': 'arxiv id of llama 3.1',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': '[2407.21783] The Llama 3 Herd of Models - arXiv.org',
   'url': 'https://arxiv.org/abs/2407.21783',
   'content': 'cs arXiv:2407.21783 arXiv author ID Modern artificial intelligence (AI) systems are powered by foundation models. This paper presents a new set of foundation models, called Llama 3. It is a herd of language models that natively support multilinguality, coding, reasoning, and tool usage. The paper also presents the results of experiments in which we integrate image, video, and speech capabilities into Llama 3 via a compositional approach. Subjects:   Artificial Intelligence (cs.AI); Computation and Language (cs.CL); Computer Vision and Pattern Recognition (cs.CV) Cite as:    arXiv:2407.21783 [cs.AI] (or arXiv:2407.21783v3 [cs.AI] for this version) Access Paper: cs.AI cs Bibliographic and Citation Tools Connected Papers Toggle Which authors of this paper a

In [47]:
print('''<function=get_arxiv_id>{"web_results": "{'query': 'arxiv id of llama 3.1', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'TheLlama3HerdofModels - arXiv.org', 'url': 'https://arxiv.org/pdf/2407.21783', 'content': 'arXiv:2407.21783v2 [cs.AI] 15 Aug 2024. Finetuned Multilingual Longcontext Tooluse Release ... The model architecture of Llama 3 is illustrated in Figure1. The development of our Llama 3 language modelscomprisestwomainstages:', 'score': 0.9955835, 'raw_content': None}, {'title': 'NousResearch/Meta-Llama-3.1-8B - Hugging Face', 'url': 'https://huggingface.co/NousResearch/Meta-Llama-3.1-8B', 'content': 'The Meta Llama 3.1 collection of multilingual large language models (LLMs) is a collection of pretrained and instruction tuned generative models in 8B, 70B and 405B sizes (text in/text out). The Llama 3.1 instruction tuned text only models (8B, 70B, 405B) are optimized for multilingual dialogue use cases and outperform many of the available ...', 'score': 0.95379424, 'raw_content': None}, {'title': 'Introducing Llama 3.1: Our most capable models to date - Meta AI', 'url': 'https://ai.meta.com/blog/meta-llama-3-1/', 'content': 'Bringing open intelligence to all, our latest models expand context length to 128K, add support across eight languages, and include Llama 3.1 405B—the first frontier-level open source AI model. Llama 3.1 405B is in a class of its own, with unmatched flexibility, control, and state-of-the-art capabilities that rival the best closed source models.', 'score': 0.9003547, 'raw_content': None}, {'title': 'The Llama 3 Herd of Models | Research - AI at Meta', 'url': 'https://ai.meta.com/research/publications/the-llama-3-herd-of-models/', 'content': 'This paper presents an extensive empirical evaluation of Llama 3. We find that Llama 3 delivers comparable quality to leading language models such as GPT-4 on a plethora of tasks. We publicly release Llama 3, including pre-trained and post-trained versions of the 405B parameter language model and our Llama Guard 3 model for input and output safety.', 'score': 0.89460546, 'raw_content': None}, {'title': '[2407.21783] The Llama 3 Herd of Models - arXiv.org', 'url': 'https://arxiv.org/abs/2407.21783', 'content': 'Modern artificial intelligence (AI) systems are powered by foundation models. This paper presents a new set of foundation models, called Llama 3. It is a herd of language models that natively support multilinguality, coding, reasoning, and tool usage. Our largest model is a dense Transformer with 405B parameters and a context window of up to 128K tokens. This paper presents an extensive ...', 'score': 0.6841585, 'raw_content': None}], 'response_time': 2.09}"}</function>''')

<function=get_arxiv_id>{"web_results": "{'query': 'arxiv id of llama 3.1', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'TheLlama3HerdofModels - arXiv.org', 'url': 'https://arxiv.org/pdf/2407.21783', 'content': 'arXiv:2407.21783v2 [cs.AI] 15 Aug 2024. Finetuned Multilingual Longcontext Tooluse Release ... The model architecture of Llama 3 is illustrated in Figure1. The development of our Llama 3 language modelscomprisestwomainstages:', 'score': 0.9955835, 'raw_content': None}, {'title': 'NousResearch/Meta-Llama-3.1-8B - Hugging Face', 'url': 'https://huggingface.co/NousResearch/Meta-Llama-3.1-8B', 'content': 'The Meta Llama 3.1 collection of multilingual large language models (LLMs) is a collection of pretrained and instruction tuned generative models in 8B, 70B and 405B sizes (text in/text out). The Llama 3.1 instruction tuned text only models (8B, 70B, 405B) are optimized for multilingual dialogue use cases and outperform many of the available ...'